In [1]:
from video_llama.common.config import Config
from video_llama.common.dist_utils import get_rank
from video_llama.common.registry import registry
from video_llama.conversation.conversation_video import Chat, Conversation, default_conversation,SeparatorStyle,conv_llava_llama_2

#%%
# imports modules for registration
from video_llama.datasets.builders import *
from video_llama.models import *
from video_llama.processors import *
from video_llama.runners import *
from video_llama.tasks import *

import os

os.chdir('/srv/flash1/pputta7/projects/lm-nav')


/coc/testnvme/pputta7/mambaforge/envs/videollama/lib/python3.9/site-packages/torchvision/transforms/functional_tensor.py:5: UserWarning: The torchvision.transforms.functional_tensor module is deprecated in 0.15 and will be **removed in 0.17**. Please don't rely on it. You probably just need to use APIs in torchvision.transforms.functional or in torchvision.transforms.v2.functional.
  warnings.warn(
/coc/testnvme/pputta7/mambaforge/envs/videollama/lib/python3.9/site-packages/torchvision/transforms/_functional_video.py:6: UserWarning: The 'torchvision.transforms._functional_video' module is deprecated since 0.12 and will be removed in the future. Please use the 'torchvision.transforms.functional' module instead.
  warnings.warn(
/coc/testnvme/pputta7/mambaforge/envs/videollama/lib/python3.9/site-packages/torchvision/transforms/_transforms_video.py:22: UserWarning: The 'torchvision.transforms._transforms_video' module is deprecated since 0.12 and will be removed in the future. Please use 

In [7]:
class Args:
    def __init__(self):
        self.cfg_path = "Video-LLaMA/eval_configs/video_llama_eval_only_vl.yaml"
        self.model_type = "llama_v2"
        self.gpu_id = 0
        self.options = []

args = Args()
cfg = Config(args)

In [8]:
model_config = cfg.model_cfg
model_cls = registry.get_model_class(model_config.arch)
model = model_cls.from_config(model_config).to('cuda:{}'.format(args.gpu_id))
model.eval()
vis_processor_cfg = cfg.datasets_cfg.webvid.vis_processor.train
vis_processor = registry.get_processor_class(vis_processor_cfg.name).from_config(vis_processor_cfg)
chat = Chat(model, vis_processor, device='cuda:{}'.format(args.gpu_id))
print('Initialization Finished')

Loading VIT
Loading VIT Done
Loading Q-Former


Using pad_token, but it is not set yet.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Load first Checkpoint: ckpt/VL_LLaMA_2_7B_Finetuned.pth
Initialization Finished


In [69]:
chat_state = conv_llava_llama_2.copy()
chat_state.system =  "You are able to understand the visual content that the user provides.\
You are a navigational robot with egocentric vision that chooses the next action by choosing from {LEFT, RIGHT, FORWARD}.\
At every step, you receive a video from the previous action. You should describe what you see from the \
visual content and then provide the next action. You are looking for a washing machine. Apply STOP when you see it."
img_list = []
llm_message = chat.upload_video_without_audio("output.mp4", chat_state, img_list)

output.mp4


In [70]:
user_message = "Describe what happened briefly."
chat.ask(user_message, chat_state)

In [71]:
llm_message = chat.answer(conv=chat_state,
                              img_list=img_list,
                              num_beams=1,
                              temperature=0.6,
                              max_new_tokens=600,
                              max_length=2000)[0]
print(chat_state.get_prompt())

[INST] <<SYS>>
You are able to understand the visual content that the user provides.You are a navigational robot with egocentric vision that chooses the next action by choosing from {LEFT, RIGHT, FORWARD}.At every step, you receive a video from the previous action. You should describe what you see from the visual content and then provide the next action. You are looking for a washing machine. Apply STOP when you see it.
<</SYS>>

<Video><ImageHere></Video> The video contains 8 frames sampled at 0.0, 0.1, 0.2, 0.4, 0.5, 0.6, 0.8, 0.9 seconds.  Describe what happened briefly. [/INST] Sure! In the video, we see a room with a white door, a white wall, and a window. There is a white fridge and a white table in the room as well. The room has a mix of vintage and modern decor, with old photographs and a clock on the wall. We can also see a guitar on the wall and a white chair in the room. The room is relatively small, and we can see a small table with a lamp on it. </s>


In [72]:
user_message = "Should you take a left, right, or go forward?"
chat.ask(user_message, chat_state)
llm_message = chat.answer(conv=chat_state,
                              img_list=img_list,
                              num_beams=1,
                              temperature=0.8,
                              max_new_tokens=700,
                              max_length=2000)[0]
print(chat_state.get_prompt())

[INST] <<SYS>>
You are able to understand the visual content that the user provides.You are a navigational robot with egocentric vision that chooses the next action by choosing from {LEFT, RIGHT, FORWARD}.At every step, you receive a video from the previous action. You should describe what you see from the visual content and then provide the next action. You are looking for a washing machine. Apply STOP when you see it.
<</SYS>>

<Video><ImageHere></Video> The video contains 8 frames sampled at 0.0, 0.1, 0.2, 0.4, 0.5, 0.6, 0.8, 0.9 seconds.  Describe what happened briefly. [/INST] Sure! In the video, we see a room with a white door, a white wall, and a window. There is a white fridge and a white table in the room as well. The room has a mix of vintage and modern decor, with old photographs and a clock on the wall. We can also see a guitar on the wall and a white chair in the room. The room is relatively small, and we can see a small table with a lamp on it. </s><s>[INST] Should you ta

In [59]:
llm_message = chat.upload_video_without_audio("output.mp4", chat_state, img_list)

output.mp4


In [60]:
user_message = "Describe the next scene"
chat.ask(user_message, chat_state)
llm_message = chat.answer(conv=chat_state,
                              img_list=img_list,
                              num_beams=1,
                              temperature=0.8,
                              max_new_tokens=700,
                              max_length=2000)[0]
print(chat_state.get_prompt())

[INST] <<SYS>>
You are able to understand the visual content that the user provides.You are a navigational robot with egocentric vision that chooses the next action by choosing to move towards an object in its vision.At every step, you receive a video from the previous action. You should describe what you see from thevisual content and then provide the next action. You are looking for a washing machine. Apply STOP when you see it.
<</SYS>>

<Video><ImageHere></Video> The video contains 8 frames sampled at 0.0, 0.1, 0.2, 0.4, 0.5, 0.6, 0.8, 0.9 seconds.  Describe what happened briefly. [/INST] First, I see a room with a white door and several pictures hanging on the wall. There is also a white chair in the room. Next, I see a room with a white door and a picture hanging on the wall. The room is decorated with a vase on the coffee table. Then, I see a room with a white door and a picture hanging on the wall. The room has a vase on the coffee table and a white chair. After that, I see a r

In [61]:
user_message = "Should you take a left, right, or go forward?"
chat.ask(user_message, chat_state)
llm_message = chat.answer(conv=chat_state,
                              img_list=img_list,
                              num_beams=1,
                              temperature=0.8,
                              max_new_tokens=700,
                              max_length=2000)[0]
print(chat_state.get_prompt())

[INST] <<SYS>>
You are able to understand the visual content that the user provides.You are a navigational robot with egocentric vision that chooses the next action by choosing to move towards an object in its vision.At every step, you receive a video from the previous action. You should describe what you see from thevisual content and then provide the next action. You are looking for a washing machine. Apply STOP when you see it.
<</SYS>>

<Video><ImageHere></Video> The video contains 8 frames sampled at 0.0, 0.1, 0.2, 0.4, 0.5, 0.6, 0.8, 0.9 seconds.  Describe what happened briefly. [/INST] First, I see a room with a white door and several pictures hanging on the wall. There is also a white chair in the room. Next, I see a room with a white door and a picture hanging on the wall. The room is decorated with a vase on the coffee table. Then, I see a room with a white door and a picture hanging on the wall. The room has a vase on the coffee table and a white chair. After that, I see a r